In [ ]:
#include <iostream>
#include <vector>
#include <algorithm>           // Include for std::max_element
#include <cmath>               // Include for std::sqrt function
#include <map>
#include <ROOT/RDataFrame.hxx> // For modern data frame tool
#include <TH1F.h>
#include <TH1I.h>
#include <TCanvas.h>

In [13]:
// Open the ROOT file containing the tree.
    TFile *file = TFile::Open("../Data/run6578.root");
    if (!file || file->IsZombie()) { // Check if the file is opened successfully.
        std::cerr << "Error opening file or file not found." << std::endl;
        return;
    }

    // Retrieve the tree named "raw" from the file.
    TTree *tree = (TTree*)file->Get("raw");
    if (!tree) { // Check if the tree exists.
        std::cerr << "Tree 'raw' not found in file." << std::endl;
        file->Close(); // Close the file if the tree is not found.
        return; // Exit the function.
    }

## Definitions

In [14]:
// Create a data frame from the TTree for easier processing.
ROOT::RDataFrame df(*tree);

std::vector<int> ids = {8,9,10,11,12,13};

// Map to store a histogram for each detector ID
std::map<int, TH1F*> histogramsQ;
std::map<int, TH1F*> histogramsTQ;
std::map<int, TH1I*> histogramsHits;

for (int id : ids) {
    histogramsHits[id] = new TH1I(Form("hCounts_%d", id),
                                    Form("Counts of Detector %d per Event;Number of hits;Number of Events", id),
                                    10, 0, 10);

    histogramsQ[id] = new TH1F(Form("hMaxQ_%d", id),
                                Form("Maximum Charge of Detector %d;Charge (ADC);Number of Events", id),
                                200, 0, 2000);

    histogramsTQ[id] = new TH1F(Form("hSumMaxCharge_%d", id), 
                                Form("Sum of Maximum Charges for Detector %d;Charge;Number of Events", id), 
                                200, 0, 2000);
}

## Preselection

Selection of events with detector id 13 in them.

In [15]:
// Lambda function to check if detector with id 13 is activated in the event.
auto hasDetector13 = [](const std::vector<unsigned int>& ids) {
    return std::find(ids.begin(), ids.end(), 13) != ids.end();
};

// Filter the data frame to include only those events where detector 13 is activated.
auto filtered = df.Filter(hasDetector13, {"apv_id"});

## Hits

In [16]:
/**********/
/** Hits **/
/**********/

filtered.Foreach([&](const std::vector<unsigned int>& apv_id) {
    std::map<int, int> hitsCount;
    for (int id : ids) {
        hitsCount[id] = std::count(apv_id.begin(), apv_id.end(), id);
        histogramsHits[id]->Fill(hitsCount[id]);
    }
}, {"apv_id"});

// Draw and save each histogram
for (int id : ids) {
    TCanvas *canvas = new TCanvas(Form("canvas_%d", id), Form("Hits in Detector %d", id), 800, 600);
    histogramsHits[id]->Draw();
}

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_8
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_9
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_10
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_11
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_12
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_13


<table border="0" style="border-collapse: collapse;">
  <tr>
    <td style="padding:5px;"><img src="../Figures/hits_detector_8.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/hits_detector_9.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/hits_detector_10.png" width="100%"></td>
  </tr>
  <tr>
    <td style="padding:5px;"><img src="../Figures/hits_detector_11.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/hits_detector_12.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/hits_detector_13.png" width="100%"></td>
  </tr>
</table>


## Maximum charge

In [17]:
/********************/
/** Charge Maximum **/
/********************/

filtered.Foreach([&](const std::vector<unsigned int>& apv_id, const std::vector<std::vector<short>>& apv_q) {
    for (size_t i = 0; i < apv_id.size(); i++) {
        int current_id = apv_id[i];
        if (histogramsQ.count(current_id) > 0 && histogramsQ[current_id] != nullptr) {
            short maxQ = *std::max_element(apv_q[i].begin(), apv_q[i].end());
            histogramsQ[current_id]->Fill(maxQ);
        }
    }   
}, {"apv_id", "apv_q"});

// Draw and save each histogram
for (int id : ids) {
    TCanvas *canvas1 = new TCanvas(Form("canvas_%d", id), Form("Histogram of Maximum Charge for Detector %d", id), 900, 600);
    histogramsQ[id]->Draw();
}

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_8
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_9
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_10
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_11
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_12
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_13


<table border="0" style="border-collapse: collapse;">
  <tr>
    <td style="padding:5px;"><img src="../Figures/maxQ_detector_8.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/maxQ_detector_9.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/maxQ_detector_10.png" width="100%"></td>
  </tr>
  <tr>
    <td style="padding:5px;"><img src="../Figures/maxQ_detector_11.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/maxQ_detector_12.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/maxQ_detector_13.png" width="100%"></td>
  </tr>
</table>


In [18]:
/************************/
/** Maximum Charge Sum **/
/************************/

filtered.Foreach([&](const std::vector<unsigned int>& apv_id, const std::vector<std::vector<short>>& apv_q) {
    std::map<int, double> sumMaxCharges; // Map to store the sum of max charges for each ID in this event

    for (size_t i = 0; i < apv_id.size(); ++i) {
        int current_id = apv_id[i];
        if (histogramsTQ.find(current_id) != histogramsTQ.end()) { // Check if we have a histogram for this ID
            const auto& charges = apv_q[i];
            if (!charges.empty()) {
                short maxCharge = *std::max_element(charges.begin(), charges.end());
                sumMaxCharges[current_id] += maxCharge; // Add max charge to sum for this detector
            }
        }
    }

    // Fill histograms with the sum of maximum charges for each relevant detector
    for (const auto& sum : sumMaxCharges) {
        histogramsTQ[sum.first]->Fill(sum.second);
    }
}, {"apv_id", "apv_q"});

// Draw and save each histogram
    for (int id : ids) {
        TCanvas *canvas2 = new TCanvas(Form("canvas_%d", id), Form("Histogram of Sum of Maximum Charges for Detector %d", id), 900, 600);
        histogramsTQ[id]->Draw();
        
    }

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_8
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_9
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_10
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_11
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_12
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_13


<table border="0" style="border-collapse: collapse;">
  <tr>
    <td style="padding:5px;"><img src="../Figures/TotalQ_detector_8.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/TotalQ_detector_9.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/TotalQ_detector_10.png" width="100%"></td>
  </tr>
  <tr>
    <td style="padding:5px;"><img src="../Figures/TotalQ_detector_11.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/TotalQ_detector_12.png" width="100%"></td>
    <td style="padding:5px;"><img src="../Figures/TotalQ_detector_13.png" width="100%"></td>
  </tr>
</table>